In [1]:
import pandas as pd
df = pd.read_csv('/content/Womens Clothing E-Commerce Reviews.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [2]:
df.drop(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Rating', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'], axis=1, inplace=True)

df.dropna(subset=['Review Text'], axis=0, inplace=True)
df.dropna(subset=['Recommended IND'], axis=0, inplace=True)
# df['Recommended IND'].astype(str)

In [3]:
# category = pd.get_dummies(df["Recommended IND"])
# df = pd.concat([df, category], axis=1)
# df = df.drop(columns='Recommended IND')
# df.rename(columns={0: 'Negative', 1:'Positive'}, inplace=True)
df.head()

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [7]:
from sklearn.model_selection import train_test_split
review = df['Review Text'].values
label = df['Recommended IND'].values
review_latih, review_test, y_latih, y_test = train_test_split(review, label, test_size=0.2)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='-')
tokenizer.fit_on_texts(review_latih) 
tokenizer.fit_on_texts(review_test)
 
sekuens_latih = tokenizer.texts_to_sequences(review_latih)
sekuens_test = tokenizer.texts_to_sequences(review_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [12]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          80000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                20736     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 117,377
Trainable params: 117,377
Non-tr

In [16]:
num_epochs = 30
history = model.fit(padded_latih, 
                    y_latih, 
                    epochs=num_epochs, 
                    batch_size=32,
                    validation_data=(padded_test, y_test), 
                    verbose=2)

Epoch 1/30
566/566 - 31s - loss: 0.0640 - accuracy: 0.9793 - val_loss: 0.5281 - val_accuracy: 0.8680 - 31s/epoch - 55ms/step
Epoch 2/30
566/566 - 32s - loss: 0.0557 - accuracy: 0.9806 - val_loss: 0.5191 - val_accuracy: 0.8680 - 32s/epoch - 56ms/step
Epoch 3/30
566/566 - 31s - loss: 0.0561 - accuracy: 0.9816 - val_loss: 0.4969 - val_accuracy: 0.8726 - 31s/epoch - 55ms/step
Epoch 4/30
566/566 - 31s - loss: 0.0462 - accuracy: 0.9849 - val_loss: 0.6003 - val_accuracy: 0.8563 - 31s/epoch - 55ms/step
Epoch 5/30
566/566 - 32s - loss: 0.0461 - accuracy: 0.9845 - val_loss: 0.5227 - val_accuracy: 0.8547 - 32s/epoch - 56ms/step
Epoch 6/30
566/566 - 33s - loss: 0.0383 - accuracy: 0.9874 - val_loss: 0.5788 - val_accuracy: 0.8669 - 33s/epoch - 59ms/step
Epoch 7/30
566/566 - 32s - loss: 0.0292 - accuracy: 0.9907 - val_loss: 0.6944 - val_accuracy: 0.8541 - 32s/epoch - 56ms/step
Epoch 8/30
566/566 - 32s - loss: 0.0328 - accuracy: 0.9895 - val_loss: 0.7004 - val_accuracy: 0.8629 - 32s/epoch - 56ms/step
